In [8]:
import numpy as np 
import pandas as pd 
import sqlalchemy as db
import psycopg2
from dotenv import load_dotenv
import os

In [14]:
earthquake_weather = pd.read_csv('../additional datasets/terremotos.csv')

In [15]:
volcano = pd.read_csv('../additional datasets/volcanes.csv')

In [16]:
tsunami = pd.read_csv('../additional datasets/tsunamis.csv')

In [3]:
earthquake = pd.read_parquet('../USGS_transformed2.parquet', engine='pyarrow')

In [14]:
poblacion = pd.read_excel('../additional datasets/poblacion_chile.xlsx')

In [17]:
#Declare globar variable - credentials in .env to make it secure
load_dotenv()

database_username= os.getenv('DATABASE_USERNAME')
database_password= os.getenv('DATABASE_PASSWORD')
database_ip= os.getenv('DATABASE_IP')
database_name=os.getenv('DATABASE_NAME')

In [10]:
poblacion = poblacion.reset_index(level=0)
poblacion['id'] = poblacion['index']

In [13]:
poblacion.rename(columns = {'index':'id'}, inplace = True)

,index,Nombre,Provincia,Región,Superficie_km2,Poblacion,Densidad_hab_km2,IDH_2005,IDH_tipo_2005,Latitud,Longitud,id
0,0,Arica,Arica,Arica y Parinacota,"4.799,4",247.552,51.6,0.866,Alto,-18.455000,-70.290000,0
1,1,Camarones,Arica,Arica y Parinacota,3.927,1.233,0.31,0.791,Alto,-19.017000,-69.867000,1
2,2,Putre,Parinacota,Arica y Parinacota,"5.902,5",2.515,0.43,0.817,Alto,-18.200000,-69.583000,2
3,3,Iquique,Iquique,Tarapacá,"2.242,1",223.463,"99,6",0.826,Alto,-20.244000,-70.139000,3
4,4,Alto Hospicio,Iquique,Tarapacá,572.9,129.999,"226,8",0.735,Medio,-20.257000,-70.022000,4
...,...,...,...,...,...,...,...,...,...,...,...,...
340,340,Talagante,Talagante,Metropolitana de Santiago,126,81.838,649.5,0.749,Alto,-33.667000,-70.931000,340
341,341,El Monte,Talagante,Metropolitana de Santiago,118,40.014,339.1,0.688,Medio,-33.684000,-71.017000,341
342,342,Isla de Maipo,Talagante,Metropolitana de Santiago,189,40.171,212.5,0.724,Alto,-33.754000,-70.886000,342
343,343,Padre Hurtado,Talagante,Metropolitana de Santiago,80.8,74.188,915.9,0.728,Alto,-33.576000,-70.800000,343


In [21]:
def initial_load(df, table_name):
    '''
    This function make the initial load into a postgress database. 
    The args would be a dataframe and a table name that should be pass as string
    '''
    #database_conection = db.create_engine(f"mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}")
    database_conection=db.create_engine(f"postgresql://{database_username}:{database_password}@{database_ip}/{database_name}")
    connection = database_conection.raw_connection()
    #cursor = connection.cursor()
    metadata=db.MetaData()
    try:
        df.to_sql(table_name, database_conection, index=False, if_exists='replace')
        print("conection open")
        print(f'Initial table {table_name} was created')
    except Exception as e:
        print("Has been a error with database conection")
        print(e)

    connection.close()
    print("The conection has been closed")

In [22]:
#load additional datasets
initial_load(earthquake_weather, 'earthquake_weather')
initial_load(volcano, 'volcano')
initial_load(tsunami, 'tsunami')

conection open
Initial table earthquake_weather was created
The conection has been closed
conection open
Initial table volcano was created
The conection has been closed
conection open
Initial table tsunami was created
The conection has been closed


In [10]:
#load main dataset
initial_load(earthquake, 'earthquake')

conection open
Initial table earthquake was created
The conection has been closed


In [6]:
initial_load(poblacion, 'poblacion_chile')

conection open
Initial table poblacion_chile was created
The conection has been closed
